In [1]:
from __future__ import print_function

In [2]:
import sys
sys.path.append('../build/')

In [3]:
import time
import copy

In [4]:
%pylab inline
np.set_printoptions(precision=4, suppress=True)

Populating the interactive namespace from numpy and matplotlib


`%matplotlib` prevents importing * from pylab and numpy


In [5]:
import versor as vsr

In [6]:
from pythreejs import *
from pythreejs import Color as Color2
import pythreejs
import numpy as np
from IPython.display import display
from ipywidgets import HTML, Text, interact
from traitlets import link, dlink

In [7]:
motor0 = vsr.Mot(1,0,0,0,0,0,0,0)
motor1 = vsr.Vec(1,1,1).trs() * vsr.Rot(vsr.Biv(0,1,0) * np.pi/6.0)
print(motor1)

Mot: [ 0.87 0 -0.5 0 -0.68 -0.43 -0.18 -0.25 ]


In [8]:
def show_line(line, color):
    v = np.array(l.loc(vsr.Vec(0,0,0).null()))[:3]
    d = np.array(l.dir())
    linesgeom = PlainGeometry(vertices=[(v + d).tolist(), (v - d).tolist()],
                                        colors = [color, color])
    return Line(geometry=linesgeom, 
                material=LineBasicMaterial(linewidth=2, vertexColors='VertexColors'), 
                type='LinePieces')

In [9]:
interact(lambda x: x**2, x=10)

100

<function __main__.<lambda>>

In [10]:
text.position = [1, 0, 0]
text.scale = [0.1, 0.1, 0.1]

In [11]:
def motor_ratio(a,b,n):
    return ((b * a.inv()).log() * (1. / n)).exp()

In [12]:
from motor_estimation import MotorEstimationSolver

In [13]:
lines_a_ = [vsr.Dll(vsr.Vec(*np.random.normal(0.0,0.8,3)),
                   vsr.Vec(*np.random.normal(0.0,0.8,3)))
          for i in range(3)]
lines_b_ = [line.spin(motor1) for line in lines_a_]
initial_motor = vsr.Mot(1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0)
mes = MotorEstimationSolver(initial_motor)
for a, b in zip(lines_a_, lines_b_):
    mes.add_line_correspondences_residual_block(a,b)
#     mes.add_line_angle_distance_residual_block(a,b)
mes.set_parameterization('BIVECTOR_GENERATOR')
mes.linear_solver_type = "DENSE_QR"
estimated_motor, lin_summary_1, motors = mes.solve()
print(lin_summary_1['brief_report'])

Ceres Solver Report: Iterations: 6, Initial cost: 1.713940e+01, Final cost: 1.666144e-25, Termination: CONVERGENCE


In [15]:
# On windows, linewidth of the material has no effect
size = 1
csys = Line(geometry=PlainGeometry(vertices=[[0, 0, 0],
                                             [size, 0, 0],
                                             [0, 0, 0],
                                             [0, size, 0],
                                             [0, 0, 0],
                                             [0, 0, size]],
                                   colors = ['red', 'red', 'green', 'green', 'blue', 'blue']), 
            material=LineBasicMaterial(linewidth=2, vertexColors='VertexColors'), 
            type='LinePieces')


lines_bs = [[line.spin(motor) for line in lines_a_] for motor in motors]
lines_a = [show_line(l, 'cyan') for l in lines_a_]
lines_b = [show_line(l, 'yellow') for l in lines_b_]
plane = Mesh(geometry = PlaneGeometry(width=2), material=PhongMaterial(color='magenta'))
scene = Scene(children=lines_a + lines_b +
              [AxisHelper(),                     
               plane,
               DirectionalLight(color='#ccaabb', position=[0,1,1]),
               AmbientLight(color='#cccccc')])

c = PerspectiveCamera(position=[3, 3, 0])
renderer = Renderer(camera=c, scene = scene, controls=[TrackballControls(controlling=c)],
                    background = 'black', renderer_type='canvas')

display(renderer)

In [33]:
m = motors[0]
for i in range(len(motors)-1):
    n = 50
    m0 = motor_ratio(motors[i], motors[i+1], float(n))
    m = motors[i]
    
    for i in range(n):
        m = m * m0
        for l in lines_a:
            l.quaternion_from_rotation(m.matrix()[:3,:3].T.flatten().tolist())
            l.position = m.matrix()[:3,3].tolist()
        time.sleep(0.001)
    time.sleep(1.)

In [16]:
x = 0.0
def f(x):
    plane.position = [0,0,x] 
interact(f, x=0.5)

In [34]:
n_planes = 10
planes_a = []
for i in range(10):
    dir_vec = np.random.normal(0.0, 0.8, 4)
    dir_vec = dir_vec / np.linalg.norm(dir_vec)
    distance = np.random.normal(0.0, 0.8,1)
    arr = dir_vec + distance
    planes_a.append(vsr.Dlp(dir_vec[0],dir_vec[1],dir_vec[2],distance[0]))
planes_b = [plane.spin(motor) for plane in planes_a]

In [35]:
n_points = 10
sigma = 0.09
points_a = [vsr.Vec(*np.random.normal(0.0, 0.8, 3)).null() 
            for i in range(n_points)]
points_b = [point.spin(motor1) for point in points_a]
points_b_noisy = [vsr.Vec(*(np.array(point)[:3] 
                            + sigma * np.random.randn(3))).null() 
                  for point in points_b]

In [36]:
initial_motor = vsr.Mot(1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0)
mes = MotorEstimationSolver(initial_motor)
for a, b in zip(points_a, points_b_noisy):
    mes.add_point_correspondences_residual_block(a,b)
mes.set_parameterization('BIVECTOR_GENERATOR')
mes.linear_solver_type = 'DENSE_QR'
(estimated_motor, summary, motors) = mes.solve()

In [37]:
# size = 1
# csys = Line(geometry=PlainGeometry(vertices=[[0, 0, 0],
#                                              [size, 0, 0],
#                                              [0, 0, 0],
#                                              [0, size, 0],
#                                              [0, 0, 0],
#                                              [0, 0, size]],
#                                    colors = ['red', 'red', 'green', 'green', 'blue', 'blue']), 
#             material=LineBasicMaterial(linewidth=2, vertexColors='VertexColors'), 
#             type='LinePieces')

plane = Mesh(geometry = PlaneGeometry(width=2), material=PhongMaterial(color='magenta'), position=[0,0,1])
center = Mesh(geometry=SphereGeometry(radius=0.05), material=LambertMaterial(color='hotpink'))
circle = Mesh(geometry = RingGeometry(thetaSegments=32, innerRadius=0.5, outerRadius=0.51), material=PhongMaterial(color='magenta'))
# _points_a = [Mesh(geometry=SphereGeometry(radius=0.05), 
#                material=LambertMaterial(color='hotpink'), 
#                position=np.array(point)[:3].tolist()) for point in 
#           [point.spin(estimated_motor) for point in points_a]]
_points_b = [Mesh(geometry=SphereGeometry(radius=0.05), 
               material=LambertMaterial(color='#6CC417'), 
               position=np.array(point)[:3].copy().tolist()) for point in points_b]
_points_a = [Mesh(geometry=SphereGeometry(radius=0.05), 
               material=LambertMaterial(color='hotpink'), 
               position=np.array(point)[:3].copy().tolist()) for point in points_a]

axishelper = AxisHelper(size=4)

# scene = Scene(children = [axishelper])

scene = Scene(children =  _points_a + _points_b + [axishelper,
               DirectionalLight(color='#ccaabb', position=[0,1,1]),
               AmbientLight(color='#cccccc')])

c = PerspectiveCamera(position=[3, 3, 0])
renderer = Renderer(camera=c, scene = scene, controls=[TrackballControls(controlling=c)],
                    background = '#eeeeee')

display(renderer)

In [21]:
n = 120
m0 = motor_ratio(initial_motor, estimated_motor, float(n))
m = initial_motor
for i in range(n):
    m = m * m0
    for a, b in zip(points_a, _points_a):
        b.position = np.array(a.spin(m))[:3].tolist()
    time.sleep(1. / 60.)

In [22]:
AxisHelper

pythreejs.pythreejs.AxisHelper

In [23]:
_points_a

In [24]:
circle.quaternion_from_rotation(motor1.matrix()[:3,:3].T.flatten().tolist())
circle.position = motor1.matrix()[:3,3].tolist()
center.position = motor1.matrix()[:3,3].tolist()

In [25]:
n = 2000
m0 = motor_ratio(initial_motor, estimated_motor, n)
m = motors[1]
for i in range(n):
    m = m * m0
print(m)
# m = motors[1]
# print(motors[1])
# for i in range(n):
#     m0 = m0 * m0
#     print(m0)
#     m = m * m0
#     print(m)
# print(motors[2])

Mot: [ 0.57 0.028 -0.82 0.051 -1.3 -0.51 -0.56 -0.87 ]


In [26]:
motors[2]

Mot: [ 0.86 -0.011 -0.51 0.0075 -0.71 -0.44 -0.19 -0.26 ]

In [27]:
print(motors)

[Mot: [ 1 0 0 0 0 0 0 0 ], Mot: [ 0.91 0.058 -0.4 0.018 -0.5 -0.46 -0.47 -0.24 ], Mot: [ 0.86 -0.011 -0.51 0.0075 -0.71 -0.44 -0.19 -0.26 ], Mot: [ 0.86 -0.013 -0.52 0.00033 -0.69 -0.43 -0.17 -0.26 ], Mot: [ 0.86 -0.014 -0.52 0.00017 -0.69 -0.43 -0.17 -0.26 ]]


In [28]:
print(motor0)

Mot: [ 1 0 0 0 0 0 0 0 ]
